In [ ]:
USE {
    dependencies {
        implementation("org.jetbrains.kotlinx:dataframe:0.13.1")
        implementation("org.apache.poi:poi-ooxml:5.2.5")
    }
}


In [ ]:
import org.jetbrains.kotlinx.dataframe.*
import org.jetbrains.kotlinx.dataframe.api.*
import org.jetbrains.kotlinx.dataframe.io.*
import org.apache.poi.xssf.usermodel.XSSFWorkbook

import java.io.ByteArrayInputStream
import java.io.ByteArrayOutputStream


In [ ]:
val perl = ProcessBuilder(listOf("stat.pl", "--xlsx", "transactions.xlsx")).start()

val outputStream = ByteArrayOutputStream()
perl.inputStream.copyTo(outputStream)
perl.waitFor()
println(if (perl.exitValue() == 0) "Stat calculated" else "Stat calculating error")

val stat = DataFrame.readCSV(ByteArrayInputStream(outputStream.toByteArray()), parserOptions = ParserOptions(nullStrings = setOf("-")))
stat


In [ ]:
File("stat.xlsx").outputStream().use { output ->
    File("statTemplate.xlsx").inputStream().use { input ->
        XSSFWorkbook(input).use { workbook ->
            workbook.removeSheetAt(workbook.getSheetIndex("data"))
            stat.writeExcel(workbook, sheetName = "data")
            workbook.write(output)
        }
    }
}
